# Variants
- Variant values are either a constant constructor, or a non-constant constructor applied to a number of values. The former case is written constr; the latter case is written constr (v1, ... , vn ), where the vi are said to be the arguments of the non-constant constructor constr. The parentheses may be omitted if there is only one argument.
- The current implementation limits each variant type to have at most 246 non-constant constructors and 230−1 constant constructors.
- The following constants are treated like built-in constant constructors:
|Constant   |Constructor|
|-----------|-----------|
|false| the boolean false|
|true| the boolean true|
|()|the "unit" value|
|[]|the empty list|
   

In [6]:
type fruits = Banana | Apple | Avocado ;;
let eat f = match f with Avocado -> "yeah!" | _ -> "nah!" ;; 
eat Avocado ;;

type peano = Zero | Succ of peano ;;

let zero = Zero ;;
let one = Succ zero ;;
let two = Succ one ;;

let count p =
    let rec loop p c =
        match p with
        | Zero -> c
        | Succ x -> loop x (c + 1) 
    in
    loop p 0 ;;

count (Succ two) ;;
count (Succ (Succ (two))) ;;

type fruits = Banana | Apple | Avocado


val eat : fruits -> string = <fun>


- : string = "yeah!"


type peano = Zero | Succ of peano


val zero : peano = Zero


val one : peano = Succ Zero


val two : peano = Succ (Succ Zero)


val count : peano -> int = <fun>


- : int = 3


- : int = 4


Variants with multiple arguments look an awful lot like tuples. But they are not. In `Pair (10, "hello")` below, `(10, "hello")` is not a 2-tuple.<br/>
The differences between a multi-argument variant and a variant containing a tuple are mostly about performance. A multi-argument variant is a single allocated block in memory, while a variant containing a tuple requires an extra heap-allocated block for the tuple.

In [10]:
type data = 
    | Int of int
    | String of string
    | Pair of int * string ;;
open Printf ;;
let show = function 
            | Int x -> sprintf "Int %d" x 
            | String x -> sprintf "String %s" x
            | Pair (x, s) -> sprintf "Int %d, String %s" x s ;;
show (Int 10) ;;
show (String "hello") ;;
show (Pair(10, "hello")) ;;

type data = Int of int | String of string | Pair of int * string


val show : data -> string = <fun>


- : string = "Int 10"


- : string = "String hello"


- : string = "Int 10, String hello"


The subtle difference between variant multiple arguments and tuple is the later can be grabbed as a whole.

In [23]:
type point_m = Point of float * float ;;
let pm = Point (1.1, 2.2) ;;

(* this won't work *)
(* let (Point x) = pm ;;  *)

let (Point (x,y)) = pm ;;

type point_t = TupledPoint of (float * float) ;;
let pt = TupledPoint (4.4, 5.5) ;;

let (TupledPoint x) = pt ;;
x ;;
let (TupledPoint (x,y)) = pt ;;

type point_m = Point of float * float


val pm : point_m = Point (1.1, 2.2)


val x : float = 1.1
val y : float = 2.2


type point_t = TupledPoint of (float * float)


val pt : point_t = TupledPoint (4.4, 5.5)


val x : float * float = (4.4, 5.5)


- : float * float = (4.4, 5.5)


val x : float = 4.4
val y : float = 5.5


Variants with inline records are both more concise and more efficient than having variants containing references to free-standing record types, because they don’t require a separate allocated object for the contents of the variant.
<br/>
The main downside is the obvious one, which is that an inline record can’t be treated as its own free-standing object.
<br/>
https://dev.realworldocaml.org/variants.html#variants-tuples-and-parens

In [33]:
type circle = { radius : float; centerX : float; centerY : float } ;;

type shape = 
    | Square of float
    | Rectangle of { width : float; height : float }
    | Circle of circle

let check_shape s =
    match s with
    | Square x -> Printf.sprintf "Square of %F" x
    | Rectangle r -> Printf.sprintf "Rect with width: %F, height: %F" r.width r.height
    | Circle c -> Printf.sprintf "Circle at %F,%F with radius %F" c.centerX c.centerY c.radius ;;

check_shape (Rectangle {width = 100.; height = 200.}) ;;
check_shape (Circle {radius = 10.; centerX = 10.; centerY=10.}) ;;

(* but this won't work *)
let get_shape s = 
    match s with
    | Circle c -> c  (* this works cause circle is a free-standing record type  *)
    | Rectangle r -> r (*  this won't work cause r is inline record *)
    | Square x -> x ;;

type circle = { radius : float; centerX : float; centerY : float; }


type shape =
    Square of float
  | Rectangle of { width : float; height : float; }
  | Circle of circle


val check_shape : shape -> string = <fun>


- : string = "Rect with width: 100., height: 200."


- : string = "Circle at 10.,10. with radius 10."


error: compile_error

##### 🟢 &nbsp; &nbsp; Variants and Recursive Data Structures

In [41]:
type 'a expr =
 | Base of 'a
 | Const of bool
 | And of 'a expr list
 | Or of 'a expr list
 | Not of 'a expr ;;
 
let rec eval expr base_eval = 
    let eval' expr = eval expr base_eval in
    match expr with
    | Base base -> base_eval base
    | Const bool -> bool
    | And l -> List.for_all eval' l
    | Or l -> List.exists eval' l
    | Not x -> not (eval' x) ;;

let base_eval s = if String.length s > 0 then true else false ;;

let expr = Or [And [Base "hello"; Base ""]; Base ""] ;;
eval expr base_eval ;;

type 'a expr =
    Base of 'a
  | Const of bool
  | And of 'a expr list
  | Or of 'a expr list
  | Not of 'a expr


val eval : 'a expr -> ('a -> bool) -> bool = <fun>


val base_eval : string -> bool = <fun>


val expr : string expr = Or [And [Base "hello"; Base ""]; Base ""]


- : bool = false


In [97]:
type 'a tree =
    | Leaf
    | Node of 'a * 'a tree * 'a tree ;;

let rec add v t =
    match t with
    | Leaf -> Node (v, Leaf, Leaf)
    | Node (x, left, right) ->
        if (v >= x) then Node (x, left, add v right) else Node (x, add v left, right) ;;

module type ADDABLE = sig
    type t
    val add : t -> t -> t
    val zero : t
end ;;

let sum (type a) (module M : ADDABLE with type t = a) (t : a tree) =
    let rec loop t acc = 
        match t with
        | Leaf -> acc
        | Node (v, t1 , t2) -> loop t1 (M.add v (loop t2 acc)) in
    loop t M.zero ;;
    
sum (module Int) (Node (10, Node (10, Leaf, Leaf), Node (10, Leaf, Node(10, Leaf, Leaf)))) ;;
sum (module Float) (Node (10., Node (10., Leaf, Leaf), Node (10., Leaf, Node(10., Leaf, Leaf)))) ;;

let t = (Leaf |> add 3 |> add 2 |> add 1 |> add 5 |> add 6  |> add 4) ;;
sum (module Int) t ;;

type 'a tree = Leaf | Node of 'a * 'a tree * 'a tree


val add : 'a -> 'a tree -> 'a tree = <fun>


module type ADDABLE = sig type t val add : t -> t -> t val zero : t end


val sum : (module ADDABLE with type t = 'a) -> 'a tree -> 'a = <fun>


- : Int.t = 40


- : Float.t = 40.


val t : int tree =
  Node (3, Node (2, Node (1, Leaf, Leaf), Leaf),
   Node (5, Node (4, Leaf, Leaf), Node (6, Leaf, Leaf)))


- : Int.t = 21


In [89]:
type 'a expr =
    | Data of 'a
    | Add of 'a expr * 'a expr
    | Multiply of 'a expr * 'a expr
    | Variable of string ;;

module type NUM = sig
    type t 
    val zero : t
    val add : t -> t -> t
    val mul : t -> t -> t
end ;;

module MS = Map.Make(String) ;;

let rec eval : type a. (module NUM with type t = a) -> (a Map.Make(String).t) -> a expr -> a  =
    fun m env expr ->  
    let module MN = (val m : NUM with type t = a) in 
    let eval' expr = eval m env expr in 
    match expr with
    | Data d -> d
    | Add (e1, e2) -> MN.add (eval' e1)  (eval'  e2)
    | Multiply (e1, e2) -> MN.mul (eval' e1) (eval' e2)
    | Variable v -> MS.find v env ;;
    
let env = MS.(empty |> add "a" 1 |> add "b" 2 |> add "c" 3) ;;
eval (module Int : NUM with type t = Int.t) env (Add(Variable "a", Multiply (Data 2, Variable "b"))) ;;

let env = MS.(empty |> add "a" 1. |> add "b" 2. |> add "c" 3.) ;;
eval (module Float) env (Add(Variable "a", Multiply (Data 2., Variable "b"))) ;;

type 'a expr =
    Data of 'a
  | Add of 'a expr * 'a expr
  | Multiply of 'a expr * 'a expr
  | Variable of string


module type NUM =
  sig type t val zero : t val add : t -> t -> t val mul : t -> t -> t end


module MS :
  sig
    type key = String.t
    type 'a t = 'a Map.Make(String).t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val filter_map : (key -> 'a -> 'b option) -> 'a t -> 'b t
    val partition : (key -> 'a -> bool) -

val eval :
  (module NUM with type t = 'a) -> 'a Map.Make(String).t -> 'a expr -> 'a =
  <fun>


val env : int MS.t = <abstr>


- : Int.t = 5


val env : float MS.t = <abstr>


- : Float.t = 5.


##### 🟢 &nbsp; &nbsp; Extensible variant types

- Extensible variant types are variant types which can be extended with new variant constructors. Extensible variant types are defined using ... New variant constructors are added using +=.
<br/>
- Pattern matching on an extensible variant type requires a default case to handle unknown variant constructors.
<br/>https://v2.ocaml.org/manual/extensiblevariants.html

In [24]:
module Expr = struct
    type t = ..
    
    type t += Int of int
    
    type t += Float of float 
      
end ;;

type Expr.t += String of string ;;

let to_string = function
    | Expr.Int x -> Int.to_string x
    | Expr.Float x -> Float.to_string x
    | String x -> x 
    | _ -> "?" ;;


module Expr :
  sig type t = .. type t += Int of int type t += Float of float end


type Expr.t += String of string


val to_string : Expr.t -> string = <fun>


Extensible variant constructors can be rebound to a different name. This allows exporting variants from another module.

In [26]:
type Expr.t += Double = Expr.Float ;;
let d = Double 10.10 ;;

type Expr.t += Double of float


val d : Expr.t = Expr.Float 10.1


Extensible variant constructors can be declared private. As with regular variants, this prevents them from being constructed directly by constructor application while still allowing them to be de-structured in pattern-matching.

In [1]:
module B : sig
  type t = ..
  type t += private Bool of int
  type t += String of string
  val bool : bool -> t
end = struct
  type t = ..
  type t += Bool of int
  type t += String of string
  let bool p = if p then Bool 1 else Bool 0
end ;;

let inspection_works = function
    | B.Bool p -> (p = 1)
    | _ -> true ;;
    
B.String "hello" ;;

(* Error: Cannot use private constructor Bool to create values of type B.t *)
(* B.Bool 10 ;;  *)

module B :
  sig
    type t = ..
    type t += private Bool of int
    type t += String of string
    val bool : bool -> t
  end


val inspection_works : B.t -> bool = <fun>


- : B.t = B.String "hello"


Extensible variant types can be declared private. This prevents new constructors from being declared directly, but allows extension constructors to be referred to in interfaces.

In [46]:
module Msg : sig
    type t = private ..
    module Make(X : sig type t end) : sig 
        type t += Str of X.t
    end
end =  struct 
    type t =  ..
    module Make(X : sig type t end) = struct 
        type t += Str of X.t
    end
end ;;

module M = Msg.Make(struct type t = string end) ;;

match M.Str "hello" with
| M.Str x -> x 
| _ -> "" ;;

module Msg :
  sig
    type t = private ..
    module Make :
      functor (X : sig type t end) -> sig type t += Str of X.t end
  end


module M : sig type Msg.t += Str of string end


- : string = "hello"
